In [ ]:
!pip install anthropic nest_asyncio

import anthropic
import json
import asyncio
import re
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional
from IPython.display import display, HTML

# Set up your API key
ANTHROPIC_API_KEY = "your-api-key-here"  # Replace with your actual API key

@dataclass
class Claim:
    text: str
    source: str
    timestamp: datetime
    priority: int
    category: str
    location: Optional[str] = None

@dataclass
class VerificationResult:
    claim: Claim
    verdict: str
    confidence: float
    sources: List[Dict]
    evidence: List[Dict]
    last_updated: datetime

class RealTimeFactChecker:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.client = anthropic.Anthropic(api_key=api_key)

    async def verify_claim(self, claim_text: str) -> Dict:
        """Verify a claim using Claude's web search tool"""

        try:
            response = self.client.messages.create(
                model="claude-3-7-sonnet-latest",
                max_tokens=2048,
                messages=[{
                    "role": "user",
                    "content": f"""Fact-check this claim using web search: "{claim_text}"

Search for reliable sources and return ONLY a JSON object with this exact format:
{{
    "verdict": "true/false/unverified",
    "confidence": 0-100,
    "evidence": [
        {{
            "finding": "what was found",
            "source": "organization name",
            "url": "full URL"
        }}
    ]
}}"""
                }],
                tools=[{
                    "type": "web_search_20250305",
                    "name": "web_search",
                    "max_uses": 5
                }]
            )

            # Get the response text
            result_text = ""
            if response.content:
                for content in response.content:
                    if hasattr(content, 'text'):
                        result_text += content.text

            # Clean up and find JSON
            result_text = result_text.strip()

            # Find JSON block (handling code blocks and plain JSON)
            json_match = None

            # Try to find JSON in code block first
            code_block_match = re.search(r'```json\s*([\s\S]*?)\s*```', result_text)
            if code_block_match:
                json_text = code_block_match.group(1)
            else:
                # Look for JSON object
                json_match = re.search(r'\{[\s\S]*\}', result_text)
                if json_match:
                    json_text = json_match.group(0)
                else:
                    json_text = result_text

            # Parse JSON
            try:
                result_data = json.loads(json_text)

                # Process evidence
                evidence_list = []
                raw_evidence = result_data.get("evidence", [])

                for item in raw_evidence:
                    if isinstance(item, dict):
                        evidence_list.append({
                            "claim": item.get("finding", ""),
                            "source": item.get("source", "Unknown"),
                            "url": item.get("url", "")
                        })

                return {
                    "verdict": result_data.get("verdict", "unverified"),
                    "confidence": float(result_data.get("confidence", 0)) / 100.0,
                    "evidence": evidence_list,
                    "sources": [],
                    "summary": ""
                }

            except json.JSONDecodeError:
                # Fallback: extract information from text
                return self._parse_text_response(result_text)

        except Exception as e:
            print(f"Error: {str(e)}")
            return {
                "verdict": "error",
                "confidence": 0.0,
                "evidence": [{"claim": f"Error: {str(e)}", "source": "System", "url": ""}],
                "sources": [],
                "summary": "Verification failed"
            }

    def _parse_text_response(self, text):
        """Parse unstructured text response"""
        verdict = "unverified"
        confidence = 0.0
        evidence = []

        # Determine verdict
        text_lower = text.lower()
        if any(word in text_lower for word in ["false", "fake", "debunked", "hoax"]):
            verdict = "false"
            confidence = 1.0
        elif any(word in text_lower for word in ["true", "confirmed", "verified"]):
            verdict = "true"
            confidence = 1.0

        # Extract evidence
        lines = text.split('\n')
        for line in lines:
            if any(keyword in line.lower() for keyword in ['confirmed', 'debunked', 'official', 'false', 'fact']):
                # Extract URLs
                url_match = re.search(r'https?://[^\s\]]+', line)
                url = url_match.group(0) if url_match else ""

                # Extract source
                source = "Official source"
                if "PIB" in line or "Press Information Bureau" in line:
                    source = "Press Information Bureau (PIB)"
                elif "Air Force" in line:
                    source = "Indian Air Force"
                elif re.search(r'([A-Z][a-z]+ [A-Z][a-z]+)', line):
                    source_match = re.search(r'([A-Z][a-z]+ [A-Z][a-z]+)', line)
                    source = source_match.group(1)

                evidence.append({
                    "claim": line.strip(),
                    "source": source,
                    "url": url
                })

        # Ensure we have at least one evidence item
        if not evidence and text:
            evidence.append({
                "claim": text[:200] + "..." if len(text) > 200 else text,
                "source": "Web search",
                "url": ""
            })

        return {
            "verdict": verdict,
            "confidence": confidence,
            "evidence": evidence,
            "sources": [],
            "summary": ""
        }

class FactCheckDashboard:
    def __init__(self, api_key: str):
        self.fact_checker = RealTimeFactChecker(api_key)
        self.results = []

    async def check_claim(self, claim_text: str, source: str = "User Input"):
        """Check a single claim"""
        print(f"🔍 Fact-checking: {claim_text[:100]}...")

        claim = Claim(
            text=claim_text,
            source=source,
            timestamp=datetime.now(),
            priority=3,
            category="general",
            location=None
        )

        verification_data = await self.fact_checker.verify_claim(claim_text)

        result = VerificationResult(
            claim=claim,
            verdict=verification_data.get("verdict", "unverified"),
            confidence=verification_data.get("confidence", 0.0),
            sources=verification_data.get("sources", []),
            evidence=verification_data.get("evidence", []),
            last_updated=datetime.now()
        )

        self.results.append(result)
        print(f"✓ Completed: {result.verdict.upper()} ({result.confidence*100:.0f}% confidence)")
        return result

async def quick_check_with_citations(claim):
    """Quick fact check with properly parsed citations"""
    dashboard = FactCheckDashboard(ANTHROPIC_API_KEY)
    result = await dashboard.check_claim(claim)

    # Color coding
    verdict_colors = {
        "true": "\033[92m",
        "false": "\033[91m",
        "unverified": "\033[93m",
        "error": "\033[91m"
    }
    color = verdict_colors.get(result.verdict.lower(), "\033[0m")
    reset = "\033[0m"

    print(f"\n{color}{'='*70}{reset}")
    print(f"Claim: {claim}")
    print(f"Verdict: {color}{result.verdict.upper()}{reset}")
    print(f"Confidence: {result.confidence*100:.0f}%")

    if result.evidence:
        print("\nEvidence with sources:")
        for i, ev in enumerate(result.evidence[:5], 1):
            claim_text = ev.get('claim', '')
            source = ev.get('source', 'Unknown')
            url = ev.get('url', '')

            print(f"\n{i}. {claim_text}")
            print(f"   Source: {source}")
            if url and url != "https://www":  # Filter out incomplete URLs
                print(f"   🔗 URL: {url}")
            else:
                print(f"   ℹ️  No direct URL available")

    print(f"\n{color}{'='*70}{reset}")
    return result

# HTML display with proper formatting
def display_html_results(dashboard):
    """Display results with properly formatted HTML"""
    html_content = """
    <div style='font-family: Arial, sans-serif; max-width: 800px; margin: 20px auto;'>
        <h2 style='color: #1a237e; border-bottom: 3px solid #1a237e; padding-bottom: 10px;'>
            📊 Fact Check Results
        </h2>
    """

    for result in dashboard.results:
        verdict_color = {
            "true": "#4caf50",
            "false": "#f44336",
            "unverified": "#ff9800",
            "error": "#f44336"
        }.get(result.verdict.lower(), "#757575")

        html_content += f"""
        <div style='border: 2px solid {verdict_color}; padding: 20px; margin: 20px 0; border-radius: 10px;'>
            <h3 style='color: {verdict_color}; margin-top: 0;'>
                {result.verdict.upper()} ({result.confidence*100:.0f}% confidence)
            </h3>
            <p><strong>Claim:</strong> {result.claim.text}</p>

            <div style='margin-top: 20px;'>
                <h4>Evidence:</h4>
        """

        for i, evidence in enumerate(result.evidence, 1):
            claim_text = evidence.get('claim', '')
            source = evidence.get('source', 'Unknown')
            url = evidence.get('url', '')

            html_content += f"""
            <div style='margin: 10px 0; padding: 15px; background-color: #f5f5f5; border-left: 3px solid {verdict_color};'>
                <p style='margin: 5px 0;'><strong>{i}.</strong> {claim_text}</p>
                <p style='margin: 5px 0; color: #666;'><strong>Source:</strong> {source}</p>
            """

            if url and url.startswith('http'):
                html_content += f"""
                <p style='margin: 5px 0;'>
                    <a href='{url}' target='_blank' style='color: #1a73e8; text-decoration: none;'>
                        🔗 View source
                    </a>
                </p>
                """
            else:
                html_content += """
                <p style='margin: 5px 0; color: #999; font-style: italic;'>
                    ℹ️ Direct URL not available
                </p>
                """

            html_content += "</div>"

        html_content += """
            </div>
        </div>
        """

    html_content += "</div>"
    display(HTML(html_content))

# Usage
import nest_asyncio
nest_asyncio.apply()

# Test claim
test_claim = "Indian Air Force pilot named Shivani Singh has been captured by Pakistan"
result = asyncio.run(quick_check_with_citations(test_claim))

# For HTML display
# dashboard = FactCheckDashboard(ANTHROPIC_API_KEY)
# asyncio.run(dashboard.check_claim(test_claim))
# display_html_results(dashboard)

🔍 Fact-checking: Indian Air Force pilot named Shivani Singh has been captured by Pakistan...
✓ Completed: FALSE (100% confidence)

Claim: Indian Air Force pilot named Shivani Singh has been captured by Pakistan
Verdict: FALSE
Confidence: 100%

Evidence with sources:

1. Press Information Bureau (PIB) of India officially stated that the claim about Squadron Leader Shivani Singh being captured in Pakistan is FAKE
   Source: Press Information Bureau (PIB)
   🔗 URL: https://www.indiatvnews.com/news/india/iaf-squadron-leader-shivani-singh-not-captured-in-pakistan-pib-debunks-fake-news-2025-05-10-989608

2. The Centre (Government of India) on May 10, 2025 explicitly debunked claims that a Female Indian Air Force pilot had been captured by Pakistan
   Source: Business Standard
   🔗 URL: https://www.business-standard.com/external-affairs-defence-security/news/fact-check-iaf-female-pilot-not-captured-centre-debunks-fake-news-125051000391_1.html

3. Indian Air Force has officially confirmed that